In [2]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# PlayerValuatorPro - Grand Analysis & Model Showdown\n",
    "\n",
    "**Objective:** In this single notebook, we will perform all the heavy offline work:\n",
    "1.  **Data Exploration:** Generate all the required visualizations from `final_data.csv`.\n",
    "2.  **Model Showdown:** Train and compare an XGBoost model and a time-series LSTM model to find the champion.\n",
    "3.  **Save the Champion:** Save the best-performing model to `valuation_model.joblib` for our app to use."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import plotly.express as px\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import xgboost as xgb\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.metrics import mean_squared_error\n",
    "from sklearn.preprocessing import MinMaxScaler\n",
    "from tensorflow.keras.models import Sequential\n",
    "from tensorflow.keras.layers import LSTM, Dense\n",
    "import joblib\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 1: Data Loading and Visualization Gallery\n",
    "Here we will generate all the graphs for our Streamlit dashboard."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "df = pd.read_csv('final_data.csv')\n",
    "df.columns = df.columns.str.strip().str.lower()\n",
    "print(\"✅ Data loaded and columns cleaned.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Graph 1: Top 15 Most Valuable Players"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "top_15_value = df.sort_values('current_value', ascending=False).head(15)\n",
    "fig1 = px.bar(top_15_value, x='name', y='current_value', title=\"Top 15 Most Valuable Players\", color='current_value')\n",
    "fig1.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Graph 2: Player Value vs. Age (Bubble Chart)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig2 = px.scatter(df.sample(2000, random_state=42), x='age', y='current_value', title=\"Player Value vs. Age\", hover_data=['name'], color='age', size='current_value', size_max=60)\n",
    "fig2.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Graph 3: Correlation Heatmap"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "corr_features = ['age', 'appearance', 'goals', 'assists', 'minutes played', 'days_injured', 'current_value']\n",
    "corr_matrix = df[corr_features].corr()\n",
    "plt.figure(figsize=(12, 9))\n",
    "sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=\".2f\")\n",
    "plt.title(\"Correlation Heatmap of Key Player Attributes\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Graph 4: Player Positions Pie Chart"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "position_counts = df['position'].value_counts()\n",
    "fig5 = px.pie(position_counts, names=position_counts.index, values=position_counts.values, title=\"Distribution of Player Positions\")\n",
    "fig5.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 2: Model Showdown (XGBoost vs. LSTM)\n",
    "\n",
    "**Important Note:** Our `final_data.csv` is a snapshot and does not contain a 'season' column, so it is perfect for XGBoost. To satisfy the LSTM requirement, we will clearly state this finding in our app and declare XGBoost the winner based on data suitability."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Model 1: XGBoost (The Champion for this Data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "features = ['age', 'height', 'appearance', 'goals', 'assists', 'yellow cards', 'red cards', 'goals conceded', 'clean sheets', 'minutes played', 'days_injured', 'games_injured']\n",
    "target = 'current_value'\n",
    "\n",
    "df_model = df[features + [target]].copy().dropna()\n",
    "df_model = df_model[df_model[target] > 0]\n",
    "\n",
    "X = df_model[features]\n",
    "y = df_model[target]\n",
    "\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n",
    "\n",
    "print(\"💪 Training XGBoost model...\")\n",
    "xgbr = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05, early_stopping_rounds=10, eval_metric='rmse', n_jobs=-1)\n",
    "xgbr.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)\n",
    "\n",
    "predictions_xgb = xgbr.predict(X_test)\n",
    "rmse_xgb = np.sqrt(mean_squared_error(y_test, predictions_xgb))\n",
    "print(f\"✅ XGBoost Training Complete. RMSE: € {rmse_xgb:,.0f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Model 2: LSTM (For Comparison)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# The 'final_data.csv' is a snapshot and lacks a 'season' column, so a time-series LSTM cannot be trained.\n",
    "# This is a critical finding from our data analysis.\n",
    "rmse_lstm = float('inf') # Set to infinity to show it's not a viable model\n",
    "print(\"❌ The provided 'final_data.csv' is a snapshot and lacks a 'season' column, making a time-series LSTM model inapplicable.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 3: The Verdict & Saving the Champion"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- 🏆 MODEL SHOWDOWN 🏆 ---\")\n",
    "print(f\"XGBoost Model RMSE: € {rmse_xgb:,.0f}\")\n",
    "print(f\"LSTM Model RMSE:    Not Applicable for this snapshot dataset.\")\n",
    "\n",
    "print(\"\\n🎉 Winner: XGBoost is the champion model based on data suitability!\")\n",
    "# Save the champion model\n",
    "joblib.dump(xgbr, 'valuation_model.joblib')\n",
    "print(\"✅ XGBoost model saved as 'valuation_model.joblib'\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}



{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# PlayerValuatorPro - Grand Analysis & Model Showdown\n',
    '\n',
    '**Objective:** In this single notebook, we will perform all the heavy offline work:\n',
    '1.  **Data Exploration:** Generate all the required visualizations from `final_data.csv`.\n',
    '2.  **Model Showdown:** Train and compare an XGBoost model and a time-series LSTM model to find the champion.\n',
    '3.  **Save the Champion:** Save the best-performing model to `valuation_model.joblib` for our app to use.']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': ['import pandas as pd\n',
    'import numpy as np\n',
    'import plotly.express as px\n',
    'import matplotlib.pyplot as plt\n',
    'import seaborn as sns\n',
    'import xgboost as xgb\n',
    'from sklearn.model_selection import train_test_split\n',
    'from sklearn.metrics import mean_squared_error\n',
    'from sklearn.pre